In [ ]:
import pandas as pd


results_xgb = pd.read_csv("/content/finalresults_xgb")
results_longformer = pd.read_csv("/content/finalresults_longformer")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

threshold = 0.6
predicted_classes = (results_xgb.prediction >= threshold).astype(int)

#Plot the confusion matrix

cm = confusion_matrix(results_xgb["outcome"], predicted_classes)
class_labels = ["Unhelpful", "Helpful"]

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues,  colorbar=False)
plt.show()


In [ ]:
#Permutation test for AUROC metric (see utils.py for the function definition)
from utils import permutation_metric

auroc_, pval = permutation_metric(results_xgb["outcome"],
                                  results_xgb.prediction,
                                  roc_auc_score,
                                  side='right',
                                  n=5000)

In [ ]:
#Permutation test for AUROC metric (see utils.py for the function definition)

auroc_, pval = permutation_metric(results_xgb["outcome"],
                                  results_longformer.prediction,
                                  roc_auc_score,
                                  side='right',
                                  n=5000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score

# Compute the ROC curve and AUC for Logistic Regression
fpr_xgb, tpr_xgb, _ = roc_curve(results_xgb["outcome"], results_xgb.prediction)
auc_xgb = round(roc_auc_score(results_xgb["outcome"], results_xgb.prediction), 2)

# Compute the ROC curve and AUC for SVM
fpr_lf, tpr_lf, _ = roc_curve(results_xgb["outcome"], results_longformer.prediction)
auc_lf = round(roc_auc_score(results_xgb["outcome"], results_longformer.prediction), 2)

# Plotting both ROC curves
plt.figure(figsize=(10, 8))
plt.plot(fpr_xgb, tpr_xgb, color='darkorange', lw=1, label=f'XGB (AUC = 0.67)')
plt.plot(fpr_lf, tpr_lf, color='blue', lw=1, linestyle='--', label=f'Longformer (AUC = 0.67)')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')

# Fill the area under the curve
plt.fill_between(fpr_xgb, tpr_xgb, alpha=0.2, color='darkorange')
plt.fill_between(fpr_lf, tpr_lf, alpha=0.2, color='blue')

# Remove the coordinate system
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().tick_params(left=False, bottom=False)
plt.gca().xaxis.set_ticks_position('none')
plt.gca().yaxis.set_ticks_position('none')

# Customize the plot
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)



plt.show()

In [ ]:
from utils import delong_roc_test

delong_roc_test(results_xgb["outcome"], results_xgb.prediction, results_longformer.prediction)